In [1]:
"""import tensorflow as tf
df = tf.test.gpu_device_name()
df"""

'import tensorflow as tf\ndf = tf.test.gpu_device_name()\ndf'

In [2]:
import torch
import transformers

In [3]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
##torch.cuda.get_device_name(0)

In [4]:
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
from nltk import tokenize
from nltk.corpus import stopwords
from gensim import utils

In [5]:
def tknz(inp, token_min_len=2, token_max_len=100, lower=True):
    lowerMap = {ord(u'A'): u'a',ord(u'A'): u'a',ord(u'B'): u'b',ord(u'C'): u'c',ord(u'Ç'): u'ç',ord(u'D'): u'd',ord(u'E'): u'e',ord(u'F'): u'f',ord(u'G'): u'g',ord(u'Ğ'): u'ğ',ord(u'H'): u'h',ord(u'I'): u'ı',ord(u'İ'): u'i',ord(u'J'): u'j',ord(u'K'): u'k',ord(u'L'): u'l',ord(u'M'): u'm',ord(u'N'): u'n',ord(u'O'): u'o',ord(u'Ö'): u'ö',ord(u'P'): u'p',ord(u'R'): u'r',ord(u'S'): u's',ord(u'Ş'): u'ş',ord(u'T'): u't',ord(u'U'): u'u',ord(u'Ü'): u'ü',ord(u'V'): u'v',ord(u'Y'): u'y',ord(u'Z'): u'z'}
    inp = inp.translate(lowerMap)
    tokenizer = TurkishTokenizer.DEFAULT
    tknL = tokenizer.tokenize(inp)
    ## (not str(t.type_) == "Type.Mention") can be added!
    ls = []
    ls.append("[CLS]")
    for t in tknL:
        if (not t.content.startswith("_")) and (not str(t.type_) == "Type.URL") and (not str(t.type_) == "Type.Punctuation"):
            ls.append(utils.to_unicode(t.content))
    ls.append("[SEP]")
    return ls

In [6]:
def conc(inp):
    st = inp[0]
    for s in inp[1:]:
        st += " "
        st += s
    return st

In [7]:
tkn = TurkishTokenizer.DEFAULT
tk = tkn.tokenize("Geçen gün www.google.com gibi bir site @Mansur etiketi paylaştı.")

In [8]:
print(conc(tknz("Geçen gün www.google.com gibi bir site @Mansur etiketi paylaştı.")))

[CLS] geçen gün gibi bir site @mansur etiketi paylaştı [SEP]


In [9]:
dat = pd.read_csv("Datasets/mağaza_yorum.csv", encoding="utf-16")
dat = dat.dropna()

In [10]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11426 entries, 0 to 11428
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Görüş   11426 non-null  object
 1   Durum   11426 non-null  object
dtypes: object(2)
memory usage: 267.8+ KB


In [11]:
dlist = []
for i in range(11426):
    if len(tknz(dat.iloc[i]["Görüş"])) == 0:
        dlist.append(i)
    else:
        dat.iloc[i]["Görüş"] = conc(tknz(dat.iloc[i]["Görüş"]))

In [12]:
for i in dlist:
    dat = dat.drop(i)

In [13]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11426 entries, 0 to 11428
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Görüş   11426 non-null  object
 1   Durum   11426 non-null  object
dtypes: object(2)
memory usage: 267.8+ KB


In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [15]:
dat.Durum = le.fit_transform(dat.Durum)

In [16]:
dat.Durum  ##Olumlu = 0 // Olumsuz = 1 // Tarafsız = 2

0        1
1        2
2        0
3        1
4        1
        ..
11424    2
11425    2
11426    1
11427    2
11428    0
Name: Durum, Length: 11426, dtype: int32

In [17]:
for i in dat.Görüş:
    print(i)
    break

[CLS] ses kalitesi ve ergonomisi rezalet sony olduğu için aldım ama 4'de 1 fiyatına çin replika ürün alsaydım çok çok daha iyiydi kesinlikle tavsiye etmiyorum [SEP]


In [18]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-cased', do_lower_case=True)
#tokenized_texts = [tokenizer.tokenize((sent)) for sent in dat.Görüş]
#print ("Tokenize the first sentence:")
#print (tokenized_texts[0])

In [19]:
def prep(data):
    input_ids = []
    attention_masks = []

    for sent in data:
        #print(sent)
        encoded_sent = tokenizer.encode_plus(
            text=conc(tknz(sent)),          # Preprocess sentence
            add_special_tokens=False,       # Add `[CLS]` and `[SEP]`
            max_length=512,                 # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            return_attention_mask=True      # Return attention mask
            )

        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
    
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    return input_ids, attention_masks

In [20]:
ii_data, at_data = prep(dat.Görüş[:3])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\disco\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
#print(dat.Görüş[:3])
#print(type(dat.Görüş[0]))
ii_data = np.squeeze(ii_data)
print(ii_data)
#ids = list(ii_data.squeeze().numpy())

tensor([[    2, 76367,  1008,  ...,     0,     0,     0],
        [    2, 76367,  1008,  ...,     0,     0,     0],
        [    2, 76367,  1008,  ...,     0,     0,     0]])


In [22]:
train_X, test_X, train_Y, test_Y = train_test_split(dat.Görüş, dat.Durum, test_size=0.2, random_state=42)

In [23]:
print(test_Y)

304     1
2154    1
9472    2
4935    2
2821    1
       ..
9884    0
4731    0
3882    0
36      2
3762    0
Name: Durum, Length: 2286, dtype: int32


In [24]:
train_inputs, train_masks = prep(train_X)
test_inputs, test_masks = prep(test_X)

In [25]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(train_Y.values)
test_labels = torch.tensor(test_Y.values)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [26]:
import torch
import torch.nn as nn
from transformers import BertModel

print(torch.cuda.get_device_name())

# Create the BertClassfier class
class BertClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 3

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('dbmdz/bert-base-turkish-128k-cased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

NVIDIA GeForce GTX 1650 Ti


In [27]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):

    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [28]:
import random
import time

loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, test_dataloader=None, epochs=4, evaluation=False):
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [29]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
#train(bert_classifier, train_dataloader, test_dataloader, epochs=2, evaluation=True)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\disco\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implement

In [30]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [31]:
import pickle

with open("model.pickle","wb") as f:
    pickle.dump(bert_classifier,f)

In [32]:
strL = ["Bu arabadan nefret ediyorum.",
        "Bugün hava çok güzel.",
        "Telefon çok iyi ama bataryası kötü hissettiriyor.",
        "hizli teslimat tesekkürler.",
        "Enflasyondaki artış nedeniyle alım gücü azaldı."
        ]

strP = []
for i in strL:
  strP.append(conc(tknz(i)))

inp, att = prep(strP)
test_dataset = TensorDataset(inp, att)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)
res = []
for p in probs:
  res.append(np.argmax(p))
# Get predictions from the probabilities
threshold = 0.9
preds = np.where(probs[:, 1] > threshold, 0, 1)

# Number of tweets predicted non-negative
print(res)
print(probs)

[2, 2, 2, 2, 2]
[[0.29803964 0.29281798 0.40914235]
 [0.2461015  0.34388357 0.4100149 ]
 [0.24449897 0.32447773 0.4310234 ]
 [0.25003693 0.34627408 0.40368897]
 [0.27527645 0.2862442  0.43847933]]


In [33]:
inp, att = prep(strP)
test_dataset = TensorDataset(inp, att)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=16)
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.6
preds = np.where(probs[:, 1] > threshold, 0, 1)

# Number of tweets predicted non-negative
print(preds)

[1 1 1 1 1]


In [34]:
print(torch.cuda.get_device_name())

NVIDIA GeForce GTX 1650 Ti


In [35]:
!pip install antlr4-python3-runtime